# Osyczka and Kundu function multi-objective optimization (min).
---
Description:

- Optimization (min)
- Multi-objective (2)
- Constraints (6)
---

Minimize the equations given by:

\begin{cases}
      f_{1}\left(\mathbf{x}\right) = -(25(x_1 - 2)^2 + (x_2 - 2)^2 + (x_3 - 1)^2 + (x_4 - 4)^2 + (x_5 - 1)^2), \\
      f_{2}\left(\mathbf{x}\right) = x_1^2 + x_2^2 + x_3^2 + x_4^2 + x_5^2 + x_6^2, \\
\end{cases}

subject to:

\begin{cases}
      C_{1}\left(\mathbf{x}\right) = (x_1 + x_2 − 2) \geq 0, \\
      C_{2}\left(\mathbf{x}\right) = (6 - x_1 − x_2) \geq 0, \\
      C_{3}\left(\mathbf{x}\right) = (2 - x_2 + x_1) \geq 0, \\
      C_{4}\left(\mathbf{x}\right) = (2 − x_1 + 3x_2) \geq 0, \\
      C_{5}\left(\mathbf{x}\right) = (4 − (x_3 − 3)2 − x_4) \geq 0, \\
      C_{6}\left(\mathbf{x}\right) = ((x_5 − 3)^2 + x_6−4) \geq 0, \\
\end{cases}

where:

\begin{cases}
      0\le x_1, x_2, x_6 \le 10, \\
      1\le x_3, x_5 \le 5, \\
      0\le x_4 \le 6. \\
\end{cases}

The Pareto-optimal region is a concatenation of five regions. Every region lies on some of the constraints. However, for the entire Pareto-optimal region, $\hat{x}_4=\hat{x}_6=0$.

### First we import python libraries and set up the directory of our code.

In [1]:
import os, sys
import numpy as np
from math import fsum

PROJECT_DIR = os.path.abspath('..')
sys.path.append(PROJECT_DIR)

### Here we import all our custom GA code.

In [2]:
# Import main classes.
from pygenalgo.genome.gene import Gene
from pygenalgo.genome.chromosome import Chromosome
from pygenalgo.engines.island_model_ga import IslandModelGA

# Import Selection Operator(s).
from pygenalgo.operators.selection.linear_rank_selector import LinearRankSelector

# Import Crossover Operator(s).
from pygenalgo.operators.crossover.uniform_crossover import UniformCrossover

# Import Mutation Operator(s).
from pygenalgo.operators.mutation.random_mutator import RandomMutator

# Import Migration Operator(s).
from pygenalgo.operators.migration.clockwise_migration import ClockwiseMigration

### Define the multi-objective function, which plays also the role of the 'fitness' function.

In addition, we define a 'rand_fx', for each variable, which takes the role of the 'random()' method
of the Genes, using the required range.

In [3]:
# Multi-objective function.
def fun_OSY(individual: Chromosome, f_min: bool = True):
    
    # Set the penalty coefficient.
    rho = 50.0
    
    # Extract the gene values.
    x = [gene.value for gene in individual.genome]
        
    # Compute each objective function.
    f1 = -(25.0*(x[0] - 2.0)**2 +
                (x[1] - 2.0)**2 +
                (x[2] - 1.0)**2 +
                (x[3] - 4.0)**2 +
                (x[4] - 1.0)**2)
    
    f2 = fsum([i**2 for i in x])
    
    # Compute the constraints.
    C1 = min(0.0, (x[0] + x[1] - 2.0))**2
    C2 = min(0.0, (6.0 - x[0] - x[1]))**2
    C3 = min(0.0, (2.0 + x[0] - x[1]))**2
    C4 = min(0.0, (2.0 - x[0] + 3.0*x[1]))**2
    C5 = min(0.0, (4.0 - (x[2] - 3.0)**2 - x[3]))**2
    C6 = min(0.0, ((x[4] - 3.0)**2 + x[5] - 4.0))**2
    
    # Compute the final value.
    f_val = fsum([0.25*f1, 0.75*f2, rho*(C1 + C2 + C3 + C4 + C5 + C6)])

    # Condition for termination.
    solution_found = False

    # Assign the fitness value (check for minimization).
    fit_value = -f_val if f_min else f_val
    
    # Return the solution tuple.
    return fit_value, solution_found
# _end_def_

# Random functions (one for each variable):
rand_fx1 = lambda: np.random.uniform(0.0, 10.0)
rand_fx2 = lambda: np.random.uniform(0.0, 10.0)
rand_fx3 = lambda: np.random.uniform(1.0,  5.0)
rand_fx4 = lambda: np.random.uniform(0.0,  6.0)
rand_fx5 = lambda: np.random.uniform(1.0,  5.0)
rand_fx6 = lambda: np.random.uniform(0.0, 10.0)

Here we set the GA parameters, such as number of genes, number of chromosomes, etc. Note that in this case each gene has different random() function, because they are valid in different ranges according to the 
problem definition.

In [4]:
# Define the number of chromosomes.
N = 400

# Initial population.
population = [Chromosome([Gene(np.random.uniform(0.0, 10.0), rand_fx1),
                          Gene(np.random.uniform(0.0, 10.0), rand_fx2),
                          Gene(np.random.uniform(1.0,  5.0), rand_fx3),
                          Gene(np.random.uniform(0.0,  6.0), rand_fx4),
                          Gene(np.random.uniform(1.0,  5.0), rand_fx5),
                          Gene(np.random.uniform(0.0, 10.0), rand_fx6)], np.nan, True)
              for _ in range(N)]

# Create the IslandModelGA object that will carry on the optimization.
test_GA = IslandModelGA(initial_pop=population,
                        fit_func=fun_OSY, num_islands=4,
                        select_op=LinearRankSelector(),
                        mutate_op=RandomMutator(0.10),
                        crossx_op=UniformCrossover(0.8),
                        migrate_op=ClockwiseMigration())

### Optimization process.

Here we call the GA object (either directly, or through the method run()). We set a number of parameter, such as the maximum iterations (i.e. epochs), tolerance for the fitness convergences, etc.

In [5]:
test_GA(epochs=1000, elitism=True, f_tol=1.0e-8, allow_migration=True, adapt_probs=True)

Parallel evolution in progress with 4 islands ...
Final Avg. Fitness = -127.1064, Spread = 542.7238.
Elapsed time: 26.204 seconds.


In [6]:
x = [gene.value for gene in test_GA.best_chromosome().genome]

f1 = -(25.0*(x[0] - 2.0)**2 +
            (x[1] - 2.0)**2 +
            (x[2] - 1.0)**2 +
            (x[3] - 4.0)**2 +
            (x[4] - 1.0)**2)

f2 = fsum([i**2 for i in x])

for i, xi in enumerate(x, start=1):
    print(f"x{i}={xi:.5f}")

print(" ")
print(f"f1(x) = {f1:.5f}")
print(f"f2(x) = {f2:.5f}")

x1=5.20066
x2=1.03780
x3=1.00065
x4=0.00037
x5=1.00026
x6=0.00216
 
f1(x) = -273.02924
f2(x) = 30.12577


### End of file